# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
# Load in csv
weather_data_df = pd.read_csv("../WeatherPy/weather_data")
weather_data_df.dropna()
weather_data_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
0,male,4.17,73.51,82.40,69,5,12.75,82.40
1,la mira,18.04,-102.33,73.98,83,90,8.03,73.98
2,henties bay,-22.12,14.28,63.00,92,73,1.99,63.00
3,rikitea,-23.12,-134.97,79.14,79,100,13.82,79.14
4,cape town,-33.93,18.42,64.69,72,58,20.80,68.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [28]:
# Store latitude and longitude in locations
locations = weather_data_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_data_df["Humidity"].astype(float)


In [29]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
#Narrow down locations based on max temp, wind speed, and humidity
ideal_max_temp = weather_data_df.loc[weather_data_df['Max Temp'] < 80]
ideal_temp = ideal_max_temp.loc[ideal_max_temp['Max Temp'] > 70]
ideal_wind = ideal_temp.loc[ideal_temp['Wind Speed'] < 10 ]
ideal_places = ideal_wind.loc[ideal_wind['Humidity'] < 65]
ideal_places.count()

City           14
Latitude       14
Longitude      14
Temperature    14
Humidity       14
Cloudiness     14
Wind Speed     14
Max Temp       14
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
#create hotel_df
hotel_df = ideal_places[['City', 'Latitude', 'Longitude']]
hotel_df["Hotel Name"] = ""
hotel_df["Country"] = ""
hotel_df.head()

C:\Users\Nytes\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Nytes\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City,Latitude,Longitude,Hotel Name,Country
7,busselton,-33.65,115.33,,
175,sekoma,-24.40,23.88,,
232,san rafael,-34.62,-68.33,,
243,puerto ayora,-0.74,-90.35,,
244,mopti,14.67,-4.00,,


In [32]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    name_address = requests.get(base_url, params=params).json()

    #Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Country"] = name_address["results"][0]["vicinity"]
        
    except (KeyError, IndexError):
    
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [33]:
hotel_df

,City,Latitude,Longitude,Hotel Name,Country
7,busselton,-33.65,115.33,Stay Margaret River,"78 Wallcliffe Rd, Margaret River"
175,sekoma,-24.40,23.88,,
232,san rafael,-34.62,-68.33,Suter Petit Hotel,"Av. Hipólito Yrigoyen 3003, San Rafael"
243,puerto ayora,-0.74,-90.35,Acacia Galapagos Boutique Hotel,"C/ Floreana Y Matazarnos, Puerto Ayora"
244,mopti,14.67,-4.00,Hotel Flandre,Sevare
250,natagaima,3.62,-75.09,Bethel Bioluxury,Villavieja
290,arona,28.10,-16.68,Hotel Riu Palace Tenerife,"Urbanización La Herradura, 38670, Adeje"
303,buenos aires,-34.61,-58.38,Hotel NH Buenos Aires Florida,San Martín 839
410,werda,-25.27,23.28,Cornwall Safari Lodge,Botswana
448,bakel,14.90,-12.46,Villa La Vallée,"Route de l'hôpital, Yaguiné 2, Bakel"


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [50]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))